In [81]:
import boto3
import time

def run_query(query, database, s3_output):
    client = boto3.client('athena')
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': s3_output,
        }
    )
    return response['QueryExecutionId']

def get_results(query_id):
    client = boto3.client('athena')
    
    while True:
        response = client.get_query_execution(QueryExecutionId=query_id)
        if response['QueryExecution']['Status']['State'] == 'SUCCEEDED':
            break
        elif response['QueryExecution']['Status']['State'] == 'FAILED':
            raise Exception("Athena query failed!")
        time.sleep(2)  # adjust this or use a back-off algorithm if needed
    
    result = client.get_query_results(QueryExecutionId=query_id)
    return result

In [92]:
query = """

WITH TeamWins AS (
    SELECT 
        t.team_id,
        t.name AS team_name,
        COUNT(*) AS wins
    FROM 
        lol.teams t
    JOIN 
        lol.tournaments tour ON 1=1
    CROSS JOIN UNNEST(tour.stages) AS s(stage)
    CROSS JOIN UNNEST(stage.sections) AS sec(section)
    CROSS JOIN UNNEST(section.matches) AS m(match)
    CROSS JOIN UNNEST(match.teams) AS tm(team)
    WHERE 
        team.result.outcome = 'win' AND t.team_id = team.id
    GROUP BY 
        t.team_id, t.name
),
TeamLosses AS (
    SELECT 
        t.team_id,
        COUNT(*) AS losses
    FROM 
        lol.teams t
    JOIN 
        lol.tournaments tour ON 1=1
    CROSS JOIN UNNEST(tour.stages) AS s(stage)
    CROSS JOIN UNNEST(stage.sections) AS sec(section)
    CROSS JOIN UNNEST(section.matches) AS m(match)
    CROSS JOIN UNNEST(match.teams) AS tm(team)
    WHERE 
        team.result.outcome = 'loss' AND t.team_id = team.id
    GROUP BY 
        t.team_id
)
SELECT 
    w.team_id,
    w.team_name,
    w.wins,
    COALESCE(l.losses, 0) AS losses
FROM 
    TeamWins w
LEFT JOIN 
    TeamLosses l ON w.team_id = l.team_id
ORDER BY 
    w.wins DESC, l.losses;

"""
database = "lol"
s3_output = "s3://query-results-144/a/Dont-bill-me/"

query_id = run_query(query, database, s3_output)
result = get_results(query_id)

In [93]:
team_dat = []
headers = []

for i, Rows in enumerate(result['ResultSet']['Rows']):
    if i == 0:
        for El in Rows['Data']:
            val = El['VarCharValue']
            headers.append(val)
        continue
 
    # for j, El in enumerate(Rows['Data']):
    #     val = El['VarCharValue']
    
    team_dat.append({f'{header}':Rows['Data'][i]['VarCharValue'] for i,header in enumerate(headers)})
      
        # print(VarCharValue['Data'][)

In [103]:
# team_dat

In [95]:
teams = dict()

for team in team_dat:
    teams[team['team_id']] = {
        'name': team['team_name'],
        'wins': team['wins'],
        'losses': team['losses']
    }

In [85]:
import pandas as pd

df_teams = pd.DataFrame.from_dict(teams, orient='index')

df_teams['wins'] = df_teams['wins'].astype(int)
df_teams['losses'] = df_teams['losses'].astype(int)

df_teams['games_played'] = df_teams['wins'] + df_teams['losses']

filtered_teams = df_teams[df_teams['games_played'] > 100]

filtered_teams

,name,wins,losses,games_played
98767991860392497,TSM,182,192,374
98767991877340524,Cloud9,156,79,235
98767991853197861,T1,153,56,209
100205573495116443,Gen.G,149,48,197
100725845018863243,Dplus Kia,142,63,205
...,...,...,...,...
103535282113853330,5 Ronin Akademi,31,75,106
105505619546859895,OKSavingsBank BRION,30,80,110
99566406069969565,Info Yatırım Aurora,29,83,112
105521419784127190,Komil&Friends,27,75,102


In [86]:
t = df_teams.copy()

In [87]:
t.loc[:, 'win_rate'] = t['wins'] / t['games_played']
t.loc[:, 'loss_rate'] = t['losses'] / t['games_played']
t.loc[t['losses'] == 0, 'win_loss_ratio'] = t['wins']
t.loc[t['losses'] != 0, 'win_loss_ratio'] = t['wins'] / t['losses']
t.loc[:, 'win_loss_difference'] = t['wins'] - t['losses']
t.loc[:, 'consistency'] = (t['wins'] - t['games_played']/2).abs()
t.loc[:, 'efficiency'] = t['wins'] / t['games_played']
t.loc[:, 'dominance'] = t['wins'] - t['losses']

In [98]:
# Defining the weights for each metric
weights = {
    'win_rate': 0.3,
    'loss_rate': -0.2,
    'win_loss_ratio': 0.25,
    'win_loss_difference': 0.15,
    'consistency': -0.1,
    'efficiency': 0.2
}

# Calculating the weighted score for each team
t_filtered.loc[:, 'weighted_score'] = sum(t_filtered[col] * weight for col, weight in weights.items())

# Sorting teams based on the weighted score in descending order to get the global ranking
global_ranking = t_filtered.sort_values(by='weighted_score', ascending=False)


/tmp/ipykernel_48/1142194386.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_filtered.loc[:, 'weighted_score'] = sum(t_filtered[col] * weight for col, weight in weights.items())


In [96]:
global_ranking.head()

,name,wins,losses,games_played,win_rate,loss_rate,win_loss_ratio,win_loss_difference,consistency,efficiency,dominance,weighted_score
100205573495116443,Gen.G,149,48,197,0.756345,0.243655,3.104167,101,50.5,0.756345,101,11.205483
98767991853197861,T1,153,56,209,0.732057,0.267943,2.732143,97,48.5,0.732057,97,10.695476
104367068120825486,PSG Talon,136,48,184,0.739130,0.260870,2.833333,88,44.0,0.739130,88,9.825725
100285330168091787,DetonatioN FocusMe,129,46,175,0.737143,0.262857,2.804348,83,41.5,0.737143,83,9.317087
100725845018863243,Dplus Kia,142,63,205,0.692683,0.307317,2.253968,79,39.5,0.692683,79,8.748370


In [102]:
t_sorted = t_filtered.sort_values(by='weighted_score', ascending=False)

ranked_teams = pd.DataFrame({
    "team_id": t_sorted.index,
    # "team_code": "TBD",  ### Need to add this to athena query
    "team_name": t_sorted["name"],
    "rank": range(1, len(t_sorted) + 1)
})
ranked_teams


,team_id,team_name,rank
100205573495116443,100205573495116443,Gen.G,1
98767991853197861,98767991853197861,T1,2
104367068120825486,104367068120825486,PSG Talon,3
100285330168091787,100285330168091787,DetonatioN FocusMe,4
100725845018863243,100725845018863243,Dplus Kia,5
...,...,...,...
105505619546859895,105505619546859895,OKSavingsBank BRION,130
99566406069969565,99566406069969565,Info Yatırım Aurora,131
101383792887446028,101383792887446028,MAMMOTH,132
98767991930907107,98767991930907107,Immortals Progressive,133


In [ ]:
import boto3
import json

def invoke_lambda(function_name, payload):
    
    lambda_client = boto3.client('lambda')
    response = lambda_client.invoke(
        FunctionName=function_name,
        InvocationType='RequestResponse',
        Payload=json.dumps(payload)
    )
    
    response_payload = json.loads(response['Payload'].read())
    return response_payload

def lambda_handler(event, context):

    team_ids = event.get('team_ids')
    if not team_ids:
        return {
            'statusCode': 400,
            'body': json.dumps({'error': 'team_ids not provided'}),
            'headers': {'Content-Type': 'application/json'}
        }

    team_games_data = {}
    for team_id in team_ids:
        games = invoke_lambda('games-by-team', {'team_id': team_id})
        team_games_data[team_id] = games

    return {
        'statusCode': 200,
        'body': json.dumps(team_games_data),
        'headers': {
            'Content-Type': 'application/json'
        }
    }